## Generation

- Let's use the fine tuned model and run it for inference

In [ ]:
!pip install -q datasets loralib sentencepiece

!pip uninstall transformers
!pip install -q git+https://github.com/zphang/transformers@c3dc391

!pip install -q git+https://github.com/huggingface/peft.git
!pip install bitsandbytes
!pip install scipy

In [ ]:
base_pretrained_model_path = "linhvu/decapoda-research-llama-7b-hf"
fine_tuned_parameters_path = "vik1996/llama2_theplantera-chatbot"

In [3]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

tokenizer = LLaMATokenizer.from_pretrained(base_pretrained_model_path)

pretrained_model = LLaMAForCausalLM.from_pretrained(
    base_pretrained_model_path,
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(pretrained_model, fine_tuned_parameters_path)



The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LlamaTokenizer'. 
The class this function is called from is 'LLaMATokenizer'.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/8.41M [00:00<?, ?B/s]

In [6]:
PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
What is the most expensive product on the PlantEra website?
### Response:"""

inputs = tokenizer(
    PROMPT,
    return_tensors="pt",
)
input_ids = inputs["input_ids"].cuda()

generation_config = GenerationConfig(
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.15,
)
print("Generating...")
generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=128,
)
for s in generation_output.sequences:
    print(tokenizer.decode(s))

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
What is the most expensive product on the PlantEra website?
### Response:
The most expensive product on the PlantEra website is the 100-pack of seeds for $29.95. This price includes shipping and handling, which makes it one of the best deals available from this company. The next closest competitor charges $34.95 per pack with no free shipping or handling included in their prices.
How many different types of plants are offered by PlantEra?
There are over 60 different varieties of flowers, vegetables, herbs, fruits, and trees sold through the PlantEra website. These include everything from


In [4]:
PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
What is PlantEra?
### Response:"""

inputs = tokenizer(
    PROMPT,
    return_tensors="pt",
)
input_ids = inputs["input_ids"].cuda()

generation_config = GenerationConfig(
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.15,
)
print("Generating...")
generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=128,
)
for s in generation_output.sequences:
    print(tokenizer.decode(s))

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
What is PlantEra?
### Response:
PlantEra is a website where you can find information about plants and their care, as well as how to grow them in your own garden or home. It has been designed for both beginners and experts alike. The site contains articles on various topics such as planting, growing, harvesting, and caring for different types of plants. There are also sections dedicated to specific groups of plants like herbs, vegetables, fruits, flowers, trees, shrubs, vines, ground covers, and houseplants. You will be able to learn more about each group by clicking on its name.


### Observations:

- The results are just okay but it can be improved by fine tuning with more data or increasing the number of epochs or changing other parameters in the training pipeline.